## "The mother ship" 
This notebook organizes inpts and function calls for creating the monte carlo csv file

##### Import functions neccessary for calling scripts

In [1]:
# I'm sure there is a better way of accessing functions 
# (perhaps just putting all in one file?); but this is what I have right now for 
# importing monte-carlo functions
from get_date import get_date
from get_lat_lon_indices import get_lat_lon_indices
from get_vessel_type import get_vessel_type
from get_vte_probability import get_vte_probability
from get_oil_capacity import get_oil_capacity
from get_length_origin_destination import get_length_origin_destination
from get_oil_type_atb import get_oil_type_atb
from get_oil_type_tanker import get_oil_type_tanker
from get_oil_type_barge import get_oil_type_barge

# import python functions needed for input variables 
import time
import yaml
import numpy
import arrow
import pathlib
from datetime import datetime, date, timedelta


# Give a path to the master yaml file that contains all the information required 
# for case selection, including file paths to data files. 

master_dir  = pathlib.Path(
    '/Users/rmueller/Projects/MIDOSS/analysis-rachael/'\
               'notebooks/monte_carlo/data/'
)
master_file = master_dir/'master.yaml'

with open(master_file) as file:
        master = yaml.load(file, Loader=yaml.Loader)
        
vessel_types = master['categories']['all_vessels']
geotiff_directory = master['directories']['geotiff']

random_seed=None
# Initialize PCG-64 random number generator
random_generator = numpy.random.default_rng(random_seed)



##### These are the input variables needed to run the functions for the monte-carlo setup

In [2]:
# spill date
start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(hours = 1)

# spill location
ais_data_year = 2018
n_locations = 1  #desired number of latitude and longitude values
upsample_factor = 1 # GeoTIFF lat/lon will be sampled at 1/upsample_factor [km] resolution


##### Imports and definitions for testing scripts and validating output

In [3]:
# these aren't used or needed for function calls
from rasterio.enums import Resampling
import rasterio as rio

#### Get the spill date

In [4]:
# get vessel traffic exposure probability by month
vte_probability = get_vte_probability( geotiff_directory )

spill_date = get_date( start_date, 
             end_date, 
             vte_probability,
             delta_time
            )
spill_month = spill_date.month
spill_month_name  = arrow.get(spill_date).format('MMMM') # string
f'{spill_month:02.0f}'

'11'

####  Get spill lat/lon value(s) as well as index values for extracting vessel time exposure values from GeoTIFFS

In [5]:
geotiff = geotiff_directory/f'all_2018_{spill_month:02.0f}.tif'


[spill_lat, spill_lon, x_index, y_index, data_val] = get_lat_lon_indices(
    geotiff_directory, spill_month, n_locations, upsample_factor
)    

Randomly selecting spill location from all-traffic GeoTIFF:
...Creating 2D array of lat/lon strings (this may take a moment)
...Selecting 1 location(s)
...Creating vectors of lat/lon pairs and x-index/y-index pairs
That's a wrap, folks!


In [6]:
spill_lat

array([-123.301])

In [7]:
spill_lon

array([48.89179])

In [8]:
x_index

array([235])

In [9]:
y_index

array([481])

In [10]:
data_val

array([19.77769089])

In [11]:
# load geotiff and access "data_val" with [x_index, y_index] to verify that the output is equivalent

traffic_reader = rio.open(
    geotiff_directory/f'all_2018_{spill_month:02.0f}.tif'
)

# dataset closes automatically using the method below

with traffic_reader as dataset:
    # resample data to target shape
    data = dataset.read(1,
        out_shape=(
            dataset.count,
            int(dataset.height * upsample_factor),
            int(dataset.width * upsample_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

print(f'compare output data_val = {data_val} hours/km^2 with {data[x_index,y_index]} hours/km^2')    

compare output data_val = [19.77769089] hours/km^2 with [19.77769] hours/km^2


####  Get vessel type 

In [12]:
vessel_type = get_vessel_type(geotiff_directory, 
                              vessel_types, 
                              ais_data_year, 
                              n_locations, 
                              spill_month,  # from get_date
                              x_index,      # from get_lat_lon_indices
                              y_index       # from get_lat_lon_indices
                              )
vessel_type

[array(['ferry'], dtype='<U9')]

#### get length origin destination
The shapefiles for this piece aren't yet done, so we need to use january and cargo

In [13]:
vessel_test = 'cargo'
month = '01'
spill_lon = -124.9243
spill_lat = 50.18442
search_radius = 0.5 #km
shapefile_directory  = f'/Users/rmueller/Data/MIDOSS/{vessel_test}_2018_{month}/'
tic = time.time()
[vessel_length, origin, destination] = get_length_origin_destination( 
    shapefile_directory, 
    vessel_test, 
    month, 
    spill_lat, 
    spill_lon, 
    search_radius 
)
toc = time.time()-tic

'this is going to take a minute....'

In [14]:
toc

44.16124391555786

In [15]:
from salishsea_tools import geo_tools

#### Determine whether spill is fuel or cargo and then determine fuel type
Note: I approach this by first determining if spill is fuel or cargo and then determining the type of fuel if it's cargo.  The caveat here is that some of the barge traffic will be determined to be fuel-spill risk only.  For the barge case only, I have set up two return variables for the get_oil_type script.  One variable is "fuel_flag".  If fuel_flag == 1 then the spill is a fuel_spill.  

The scripts for getting oil cargo type are called as follows:

- oil_type = get_oil_type_tanker(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )

- oil_type = get_oil_type_atb(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )

- [oil_type, fuel_flag] = get_oil_type_barge(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )


In [16]:
fuel_file   = master['files']['fuel']

with open(fuel_file) as file:
        fuel = yaml.load(file, Loader=yaml.Loader)

# These values don't take into consideration new insights w.r.t. tanker values
# Need to review choices
fuel

{'atb': {'bunker': 0.38, 'diesel': 0.62},
 'barge': {'bunker': 0.38, 'diesel': 0.62},
 'cargo': {'bunker': 0.97, 'diesel': 0.03},
 'cruise': {'bunker': 0.8, 'diesel': 0.2},
 'ferry': {'bunker': 0.0, 'diesel': 1.0},
 'fishing': {'bunker': 0.14, 'diesel': 0.86},
 'other': {'bunker': 0.32, 'diesel': 0.68},
 'smallpass': {'bunker': 0.0, 'diesel': 1.0},
 'tanker': {'bunker': 0.64, 'diesel': 0.36}}

In [17]:
tank_vessels = master['categories']['tank_vessels']
vessel_type = 'tanker'

if vessel_type in tank_vessels:
    
    # 
    # fuel-spill or cargo-spill?
    probability = [
        master['vessel_attributes'][vessel_type]['probability_cargo'],
        master['vessel_attributes'][vessel_type]['probability_fuel']
    ]
    
    fuel_flag = 0 #indicates a fuel-spill
    #fuel_flag = random_generator.choice(
#                     [0, 1], 
#                     p = probability)
    
    if not fuel_flag:
        if vessel_type == 'atb':
            cargo_oil_type = get_oil_type_atb(
                master_file, 
                origin, 
                destination,
                random_generator
            )
        elif vessel_type == 'barge':
            [cargo_oil_type, fuel_flag] = get_oil_type_barge( 
                master_file, 
                origin, 
                destination,
                random_generator
            )
        elif vessel_type == 'tanker':
            cargo_oil_type = get_oil_type_tanker(
                master_file, 
                origin, 
                destination,
                random_generator
            )
    # The indentation here is to catch the cases that are originally 
    # determined as fuel spills as well as cargo spills converted to fuel-spills         
    if fuel_flag:
        fuel_oil_type = random_generator.choice(
                    ['bunker', 'diesel'], 
                    p = [
                        fuel[vessel_type]['bunker'], 
                        fuel[vessel_type]['diesel']
                    ]
        )
        
# fuel-type for non-tank traffic
else:
    fuel_oil_type = random_generator.choice(
                    ['bunker', 'diesel'], 
                    p = [
                        fuel[vessel_type]['bunker'], 
                        fuel[vessel_type]['diesel']
                    ]
        )

### Calculate spill volume

In [18]:
[fuel_capacity, cargo_capacity] = get_oil_capacity(
    master_file,
    vessel_length, 
    vessel_type,
    1
)

if fuel_flag: 
    oil_type = fuel_oil_type
    oil_capacity = fuel_capacity
else:
    oil_type = cargo_oil_type
    oil_capacity = cargo_capacity


In [19]:
# test all vessel types
for vessel in vessel_types:
    [fuel_capacity, cargo_capacity] = get_oil_capacity(
    master_file,
    vessel_length, 
    vessel,
    random_generator
)

### Apply Susan's probability function to oil_capacity to get spill volume